In [ ]:
# jupyter nbconvert --to script {파일명}.ipynb
#jupyter nbconvert --to script casca_train.ipynb
# ipynb -> py
import torch
torch.cuda.empty_cache()

In [1]:
!ps -ef | grep jupyter

root       54061       1  0 23:36 ?        00:00:00 /opt/conda/bin/python -m ipykernel_launcher --f=/data/ephemeral/home/.local/share/jupyter/runtime/kernel-v2-41119VZugGs4kTOT0.json
root       54285   54083 34 23:37 ?        00:00:00 /opt/conda/bin/python -m ipykernel_launcher --f=/data/ephemeral/home/.local/share/jupyter/runtime/kernel-v2-540836ZpsSg6EBEEq.json
root       54306   54285 56 23:37 pts/2    00:00:00 /bin/bash -c ps -ef | grep jupyter
root       54308   54306  0 23:37 pts/2    00:00:00 grep jupyter


In [2]:
!kill -9 54308

/bin/bash: line 0: kill: (54308) - No such process


In [1]:
!nvidia-smi

Mon Jan  8 23:58:47 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   31C    P0    64W / 300W |  14373MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
#import wandb
import torch

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

#wandb.init(project='cascade', name='cascade641x_weighted')
# config file 들고오기

cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_1x_coco.py')
#cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_20e_coco.py')

root='../../../dataset/'

weighted_cross_entropy_loss = torch.tensor([5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]).cuda()
# 1General trash / 2Paper / 3Paper pack / 4Metal / 5Glass / 6Plastic /7Styrofoam / 8Plastic bag / 9Battery / 10Clothing
cfg.model.rpn_head.loss_cls.loss_weight = weighted_cross_entropy_loss            
cfg.model.roi_head.bbox_head[0].loss_cls.loss_weight = weighted_cross_entropy_loss
cfg.model.roi_head.bbox_head[1].loss_cls.loss_weight = weighted_cross_entropy_loss
cfg.model.roi_head.bbox_head[2].loss_cls.loss_weight = weighted_cross_entropy_loss

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()
'''
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project':'cascade'},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100)
]
'''

"\ncfg.log_config.hooks = [\n    dict(type='TextLoggerHook'),\n    dict(type='MMDetWandbHook',\n         init_kwargs={'project':'cascade'},\n         interval=10,\n         log_checkpoint=True,\n         log_checkpoint_metadata=True,\n         num_eval_images=100)\n]\n"

In [6]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [7]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-08 23:37:59,854 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://resnext101_64x4d'}
2024-01-08 23:37:59,855 - mmcv - INFO - load model from: open-mmlab://resnext101_64x4d
2024-01-08 23:37:59,856 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://resnext101_64x4d
2024-01-08 23:38:00,186 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-01-08 23:38:00,212 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-01-08 23:38:00,219 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2024-01-08 23:38:00,325 - mmcv - INFO - initialize Shared2FCBBo

In [8]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-08 23:38:00,918 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-08 23:38:00,922 - mmdet - INFO - Start running, host: root@instance-5471, work_dir: /data/ephemeral/home/level2-objectdetection-cv-10/develop/yumin/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash
2024-01-08 23:38:00,923 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_N

2024-01-08 23:38:50,471 - mmdet - INFO - Epoch [1][50/1221]	lr: 9.890e-06, eta: 10:04:03, time: 0.991, data_time: 0.056, memory: 12396, loss_rpn_cls: 0.7271, loss_rpn_bbox: 0.0501, s0.loss_cls: 1.4921, s0.acc: 74.3506, s0.loss_bbox: 0.0604, s1.loss_cls: 0.7105, s1.acc: 77.5820, s1.loss_bbox: 0.0173, s2.loss_cls: 0.4372, s2.acc: 61.6055, s2.loss_bbox: 0.0024, loss: 3.4970, grad_norm: 24.7517


KeyboardInterrupt: 